In [1]:
pip install -r requirements.txt

  Using cached tensorflow-2.16.2-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tf-keras==2.16.2 (from versions: 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1rc0, 2.15.1, 2.16.0rc0, 2.16.0rc1, 2.16.0rc2, 2.16.0rc3, 2.16.0rc4, 2.16.0, 2.17.0rc0, 2.17.0)
ERROR: No matching distribution found for tf-keras==2.16.2


In [18]:
import os
from transformers import BertTokenizer, BertModel
import torch
import faiss
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from llama_index.llms.groq import Groq
import re
import yaml
from bs4 import BeautifulSoup
import markdown
import random
import time

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Monster\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with open('auth.yaml', 'r') as config_file:
    config = yaml.load(config_file,Loader=yaml.Loader)
my_api_key = config['api_key']

In [4]:
# Regular expression pattern to match table structures
table_pattern = re.compile(r'(\|(?:.*?\|)+(?:\n\|(?:.*?\|)+)+)')
# Regular expression pattern to match sub-section headers
sub_section_pattern = r'\b\d+\.\d+\.?\d* .+'

def extract_sections_from_html(html_content):
    """
    Extracts sections from HTML content based on header tags (e.g., <h2>, <h3>).
    Args: html_content (str): The HTML content to parse.
    Returns: dict: A dictionary where keys are header texts and values are the content under each header.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = {}
    
    # Find all headers (e.g., <h2>, <h3>)
    headers = soup.find_all(['h2', 'h3'])  # Adjust tags as needed
    for header in headers:
        header_text = header.get_text(strip=True)
        # Collect content under each header until the next header is found
        content = []
        sibling = header.find_next_sibling()
        while sibling and sibling.name and not sibling.name.startswith('h'):
            content.append(sibling.get_text(separator='\n', strip=True))
            sibling = sibling.find_next_sibling()
        sections[header_text] = '\n'.join(content)
    
    return sections

def extract_tables_and_text(text):
    """
    Extracts tables from the text and separates them from the main text.
    Args: text (str): The text content containing tables.
    Returns: tuple: A tuple where the first element is the text with tables replaced by a placeholder, and the second element is a list of formatted table texts.
    """
    # Extract tables using the predefined regular expression pattern
    tables = table_pattern.findall(text)
    
    # Replace tables in the text with a placeholder
    text_without_tables = table_pattern.sub("<TABLE_PLACEHOLDER>", text)
    
    # Convert tables to a more readable structured text format
    table_texts = []
    for table in tables:
        rows = table.strip().split('\n')
        headers = rows[0].split('|')[1:-1]
        data_rows = [row.split('|')[1:-1] for row in rows[2:]]
        table_text = "Table:\n"
        table_text += " | ".join(headers) + "\n"
        for data_row in data_rows:
            table_text += " | ".join(data_row) + "\n"
        table_texts.append(table_text)
    
    return text_without_tables, table_texts

def reintegrate_tables(text, table_texts):
    """
    Reintegrates tables into the text where they were previously removed.
    Args:   text (str): The text with placeholders for tables.
            table_texts (list): List of formatted table texts to be inserted back.
    Returns:str: The text with tables reintegrated.
    """
    # Split the text by placeholders to insert tables back in their original positions
    sections = text.split("<TABLE_PLACEHOLDER>")
    combined_text = ""
    table_index = 0
    for section in sections:
        combined_text += section
        if table_index < len(table_texts):
            combined_text += table_texts[table_index]
            table_index += 1
    return combined_text

def clean_white_spaces(text):
    """
    Cleans up extra white spaces and newlines in the text.
    Args: text (str): The text to be cleaned.
    Returns:str: The cleaned text with normalized white spaces.
    """
    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)
    
    # Remove extra spaces before and after newlines
    text = re.sub(r'\s*\n\s*', '\n', text)
    
    # Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading and trailing white spaces
    text = text.strip()
    
    return text


In [5]:
def extract_and_process_text(files_folder):
    """
    Extracts and processes text from Markdown files in a specified folder, 
    splitting into sections and preserving tables.
    Args: files_folder (str): The path to the folder containing the Markdown files.
    Returns: list: A list of processed text sections.
    """
    texts = []  # Initialize an empty list to store the processed text sections
    
    # Loop through all files in the specified folder
    for file in os.listdir(files_folder):
        # Check if the current file is the desired Markdown file
        if file.endswith('conda-tutorial.md'):
            # Construct the full file path
            file_path = os.path.join(files_folder, file)
            
            # Read the content of the Markdown file
            with open(file_path, 'r', encoding='utf-8') as f:
                doc = f.read()
            
            # Convert Markdown content to HTML
            html_content = markdown.markdown(doc)
            
            # Extract sections from the HTML content
            paragraphs = extract_sections_from_html(html_content)

            # Initialize a list to hold the processed text for the current file
            combined_texts = []
            
            for k, v in paragraphs.items():
                # Reformat section headers and combine with the section content
                combined_value = re.sub(r'(\d+\.\d+\.?\d*)([A-Za-z])', r'\1 \2', k) + ' ' + v

                # Extract tables and remaining text from the combined value
                text_without_tables, table_texts = extract_tables_and_text(combined_value)
                
                # Reintegrate tables back into the main text
                processed_text = reintegrate_tables(text_without_tables, table_texts)
                
                # Clean up extra white spaces in the processed text
                sections = clean_white_spaces(processed_text)
                
                # Add the cleaned sections to the list of texts
                texts.append(sections)
    
    return texts  # Return the list of processed text sections

# Specify the folder containing the Markdown files
current_directory = os.path.dirname(os.getcwd())  # Get the directory of the current script
files_folder = os.path.join(current_directory, 'RAG-SELECT-AI', 'files')

# Call the function and store the extracted and processed texts
processed_texts = extract_and_process_text(files_folder)

In [ ]:
combined_processed_texts = ' '.join(processed_texts)

In [8]:
# Function to segment text into overlapping chunks
def segment_text_with_overlap(texts, chunk_size, chunk_overlap):
    """
    Segments the input text into chunks of a specified size with overlapping parts.
    Parameters  - texts (str): The input text to be segmented.
                - chunk_size (int): The number of words in each chunk.
                - chunk_overlap (int): The number of words that overlap between consecutive chunks.
    Returns: List of strings, where each string is a chunk of the input text.
    """
    segments = []  # List to store the resulting text chunks
    words = word_tokenize(texts)  # Tokenize the input text into a list of words
    step = chunk_size - chunk_overlap  # Calculate the step size for moving the window
    # Loop over the words to create chunks with overlap
    for i in range(0, len(words), step):
        # Create a chunk of the specified size, including the overlap
        chunk = ' '.join(words[i:i + chunk_size])
        segments.append(chunk)  # Append the chunk to the list of segments
    return segments  # Return the list of text chunks



segments = segment_text_with_overlap(combined_processed_texts, chunk_size=300, chunk_overlap=20)
# Print the third segment (index 2) for demonstration
print(f"Segment {2}:", segments[2])


Segment 2: to a tarball or bzipped tar archive , with the extension `` .tar.bz2 '' , which contains metadata under the info/ directory and a collection of files that are installed directly into an install prefix . During the install process , files are extracted into the install prefix , except for files in the info/ directory . Installing the files of a conda package into an environment can be thought of as changing the directory to an environment , and then downloading and extracting the .zip file and its dependencies—all with the single conda install [ packagename ] command . 1.3 Getting Started With Conda Conda is a powerful package manager and environment manager that you use with command line commands at the Anaconda Prompt for Windows , or in a Terminal window for macOS or Linux . This 20-minute guide to getting started with conda lets you try out the major features of conda . You should understand how conda works when you finish this guide . SEE ALSO : Getting started with Anac

In [9]:
len(segments)

82

In [10]:
# Load the BERT tokenizer and model from the pretrained 'bert-base-multilingual-cased' model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def vectorize_texts(texts, tokenizer, model):
    """
    Convert a list of texts into BERT embeddings.
    Args:   texts (list of str): A list of text strings to be vectorized.
            tokenizer (BertTokenizer): The BERT tokenizer used to preprocess the texts.
            model (BertModel): The BERT model used to generate embeddings.
    Returns:torch.Tensor: A tensor containing the embeddings of the [CLS] token for each input text.
    """
    # Tokenize the input texts
    # - return_tensors='pt': Converts tokens into PyTorch tensors.
    # - padding=True: Pads all sequences to the same length.
    # - truncation=True: Truncates sequences longer than max_length.
    # - max_length=512: Sets the maximum length of sequences to 512 tokens.
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Generate embeddings using the model
    # - torch.no_grad(): Disables gradient calculation (no backpropagation).
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract embeddings
    # - last_hidden_state: Contains hidden states of the model.
    # - [:, 0, :]: Extracts the embeddings of the [CLS] token (the first token) for each sequence.
    embeddings = outputs.last_hidden_state[:, 0, :]
    
    return embeddings

#Vectorize the list of text segments
vectors = vectorize_texts(segments, tokenizer, model)


In [11]:
# Convert the PyTorch tensor of vectors to a NumPy array
vectors_np = vectors.numpy()

# Get the dimension of the vectors (number of features)
dimension = vectors_np.shape[1]

# Create a FAISS index for fast nearest neighbor search using L2 (Euclidean) distance
index = faiss.IndexFlatL2(dimension)

# Add the vector representations to the FAISS index
index.add(vectors_np)


In [12]:
def retrieve_passages(query, tokenizer, model, index, segments, top_k=10):
    """
    Retrieve the most relevant passages from a set of segments based on a query.
    Args:   query (str): The search query text.
            tokenizer (BertTokenizer): The BERT tokenizer used to preprocess the query.
            model (BertModel): The BERT model used to generate embeddings for the query.
            index (faiss.Index): FAISS index used for similarity search.
            segments (list of str): List of text segments to search through.
            top_k (int): Number of top similar passages to retrieve.
    Returns: tuple: A tuple containing:
            - A list of the most relevant passages.
            - A numpy array of distances corresponding to the passages.
    """
    # Vectorize the query text using the BERT model
    # - Convert the query into a tensor that can be processed by the model
    query_vector = vectorize_texts([query], tokenizer, model)  # Note: Pass query as a list
    
    # Perform the similarity search in the FAISS index
    # - Search for the top_k most similar vectors to the query vector
    distances, indices = index.search(query_vector, top_k)
    
    # Retrieve the segments corresponding to the indices returned by FAISS
    # - Map indices to the actual segments
    results = [segments[idx] for idx in indices[0]]
    
    # Return the retrieved segments and their corresponding distances
    return results, distances

In [13]:
def create_response(passages):
    # Join the passages with double newlines to create the response text
    response = "\n\n".join(passages)
    return response

The LLM Model

In [14]:
def generate_answer(question):
    #Retrieve the most relevant passages and their distances based on the query
    relevant_passages,dist = retrieve_passages(query= question , tokenizer=tokenizer, model=model, index=index, segments=processed_texts, top_k=10)
    
    llm = Groq(model="llama3-70b-8192", api_key=my_api_key)

    prompt = f"""   Answer the question from the given paragraph
                    Don't repeat the question in the answer
                    if You dont find the answer in the Paragraphs say I dont know
                    make your answer easy and simple to understand
                    {question}{create_response(relevant_passages)}"""
    answer = llm.complete(prompt)
    return answer

# Print the response created from the relevant passages


In [16]:
questions = [
    "What is the primary interface for managing installations of various packages in Conda?",
    "How can you abbreviate frequently used command options in Conda?",
    "What is the ROOT_DIR in Conda's directory structure?",
    "What does the /Pkgs directory contain in Conda?",
    "What is the system location for additional Conda environments?",
    "What subdirectories are included in the default Anaconda environment?",
    "What is a Conda environment?",
    "Give an example of a Conda environment with specific packages.",
    "What is a Conda package?",
    "From where are Conda packages downloaded?",
    "How do you install a Conda package?",
    "What is included in a Conda package?",
    "What is the quickest way to start using Conda?",
    "What should you do if you get an error message after verifying Conda installation?",
    "How do you update Conda to the current version?",
    "What is the default environment in Conda called?",
    "Why should you create separate environments in Conda?",
    "How do you create a new environment and install a package in it?",
    "How do you activate a new environment in Conda?",
    "How do you list all your environments in Conda?",
    "How do you change your current environment back to the default (base) in Conda?",
    "What version of Python does Conda install by default when creating a new environment?",
    "How do you create a new environment with a specific version of Python in Conda?",
    "How do you verify the version of Python in your current environment?",
    "How do you check which packages are installed in your current Conda environment?",
    "How do you search for a package in the Anaconda repository?",
    "How do you install a package in the current environment?",
    "What command lists all installed packages in a Conda environment?",
    "What is the URL for the full documentation of Conda?",
    "What are the system requirements for Miniconda?",
    "What are the system requirements for Anaconda?",
    "What should you do if you already have other Python installations or packages and want to install Conda?",
    "How do you see which Python installation is currently set as the default on macOS or Linux?",
    "How do you see which Python installation is currently set as the default on Windows?",
    "How do you see which packages are installed in your current Conda environment?",
    "What are the three Conda download options?",
    "What is the GUI version of Conda used for?",
    "How do you verify the cryptographic hash of a downloaded installer on macOS?",
    "How do you verify the cryptographic hash of a downloaded installer on Linux?",
    "How do you verify the cryptographic hash of a downloaded installer on Windows with PowerShell V4 or later?",
    "What should you do if you don't have PowerShell V4 or later to verify the cryptographic hash?",
    "What is the command to start silent installation of Miniconda on Windows?",
    "What does the /S argument do in the Miniconda silent installation command?",
    "What does the /AddToPath argument do in the Miniconda silent installation command?",
    "What does the /RegisterPython argument do in the Miniconda silent installation command?",
    "What is the default installation type for Miniconda on Windows?",
    "How do you activate the 'snowflakes' environment on Windows?",
    "How do you activate the 'snowflakes' environment on Linux or macOS?",
    "How do you deactivate an environment on Windows?",
    "How do you deactivate an environment on Linux or macOS?"
]

In [36]:
random_question = random.choice(questions)
correct_answers = 0
counter = 0

#question ="What subdirectories are included in the default Anaconda environment?"
for question in questions:
    answer = generate_answer(question)
    print(f"{question} \n \n \n{answer}\n ------------------------------------\n")
    if ("I don't know" not in str(answer)) and ("I'm ready to answer your questions based on the provided paragraph. Go ahead and ask your question!" not in str(answer)):
        correct_answers += 1
    
    # Increment the counter
    counter += 1   
    
    # Add a sleep delay after the 10th and 20th items
    if counter % 5 == 0:
        time.sleep(15)
    time.sleep(5)

What is the primary interface for managing installations of various packages in Conda? 
 
 
The primary interface for managing installations of various packages in Conda is the conda command.
 ------------------------------------

How can you abbreviate frequently used command options in Conda? 
 
 
You can abbreviate frequently used command options in Conda by using 1 dash and the first letter of the option instead of 2 dashes.
 ------------------------------------

What is the ROOT_DIR in Conda's directory structure? 
 
 
The ROOT_DIR in Conda's directory structure is also referred to as PKGS_DIR, and it contains decompressed packages, ready to be linked in conda environments.
 ------------------------------------

What does the /Pkgs directory contain in Conda? 
 
 
The /Pkgs directory contains output packages that are moved here if the build or test succeeds.
 ------------------------------------

What is the system location for additional Conda environments? 
 
 
The system locati

In [38]:
print(f"The model answered {correct_answers} out of {len(questions)} questions correctly.")

The model answered 28 out of 50 questions correctly.
